In [1]:
from langgraph.graph import StateGraph,START,END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

In [2]:
load_dotenv()
model = ChatOpenAI()

In [3]:
class BlogState(TypedDict):
    title : str
    outline : str
    content : str

In [4]:
def create_outline(state:BlogState)->BlogState:
    # Fetch title
    title = state['title']

    # Call llm gen outline
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content

    # Update state
    state['outline']=outline

    return state

In [5]:
def create_blog(state:BlogState)->BlogState:
    title = state['title']
    outline = state['outline']
    prompt = f'Write a deatiled blog on the title - {title} using the following outline \n {outline}'
    content = model.invoke(prompt).contnet
    state['content'] = content
    return state

In [6]:
graph = StateGraph(BlogState)
# Nodes
graph.add_node('create_outline',create_outline)
graph.add_node('create_blog',create_blog)

# Edges
graph.add_edge(START,'create_outline')
graph.add_edge('create_outline','create_blog')
graph.add_edge('create_blog',END)

workflow = graph.compile()


In [ ]:
initial_state = {'title':'Rise of AI in India'}
final_state = workflow.invoke(initial_state)
print(final_state)
